# 0. インポート

In [ ]:
# pip install git+https://github.com/huggingface/peft.git

In [1]:
from google.colab import drive
drive.mount("/content/drive/")
%cd "/content/drive/MyDrive/Colab Notebooks/chat_bot"

Mounted at /content/drive/
/content/drive/MyDrive/Colab Notebooks/chat_bot


In [ ]:
!nvidia-smi

Sat Jul 20 18:18:51 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   72C    P0              31W /  70W |  14995MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!pip uninstall accelerate bitsandbytes -y
!pip install accelerate
!pip install --upgrade bitsandbytes
!pip install datasets
!pip install peft

# !pip install tiktoken
# !pip install transformers_stream_generator einops

Found existing installation: accelerate 0.32.1
Uninstalling accelerate-0.32.1:
  Successfully uninstalled accelerate-0.32.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 9.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cac

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, Trainer, TrainingArguments
from peft import LoraConfig, get_peft_model, TaskType, PeftModel

# 0.5. モデルロード

In [4]:
# load adaptor
from peft import LoraConfig, get_peft_model, TaskType
from transformers import AutoModelForCausalLM, Trainer, TrainingArguments

# peft_model_id = "tokyotech-llm/Swallow-7b-instruct-v0.1"
peft_model_id = "tokyotech-llm/Swallow-13b-instruct-v0.1" # 34分
# model = AutoModelForCausalLM.from_pretrained(peft_model_id)

# LoRAの設定
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,  # 訓練タスクの指定
    r=4,  # 訓練過程で学習される低ランク行列のランク
    lora_alpha=16,  # 適応層の出力に適用されるスケーリング値
    lora_dropout=0.1,  # 過学習を防ぐためのドロップアウト率
    bias="none",  # バイアスの使用
)

quantization_config = BitsAndBytesConfig(
        load_in_4bit = True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype = torch.bfloat16,
        bnb_4bit_use_double_quant = True
)
model = AutoModelForCausalLM.from_pretrained(
                                            peft_model_id,
                                            quantization_config=quantization_config,
                                            torch_dtype=torch.bfloat16,
                                            device_map="auto",
                                            # ignore_mismatched_sizes=True,
                                            )

tokenizer = AutoTokenizer.from_pretrained(peft_model_id)
print("completed.")

# LoRAアダプターをモデルに適用
model = get_peft_model(model, lora_config)
# model.load_adapter(peft_model_id, lora_config)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/757 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/29.9k [00:00<?, ?B/s]

model-00001-of-00006.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00006.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/203 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/914k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/457 [00:00<?, ?B/s]

completed.


In [ ]:
print(model.named_parameters)

<bound method Module.named_parameters of PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(43177, 5120)
        (layers): ModuleList(
          (0-39): 40 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=5120, out_features=5120, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=5120, out_features=4, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=4, out_features=5120, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear4bit(in_fe

In [ ]:
# # モデル訓練後にそのモデルを保存
print("Process start.")
model.save_pretrained("quantized_swallow13bIns_loraGetModel")
print("Saving the model complete.")
tokenizer.save_pretrained("quantized_swallow13bIns_loraGetModel")
print("Saveing the tokenizer complete.")

Process start.
Saving the model complete.
Saveing the tokenizer complete.


In [ ]:
# ## 約2分 ##13min

# peft_model_id = "quantized_swallow13bIns_loraGetModel"

# # LoRAの設定
# lora_config = LoraConfig(
#     task_type=TaskType.CAUSAL_LM,  # 訓練タスクの指定
#     r=4,  # 訓練過程で学習される低ランク行列のランク
#     lora_alpha=16,  # 適応層の出力に適用されるスケーリング値
#     lora_dropout=0.1,  # 過学習を防ぐためのドロップアウト率
#     bias="none",  # バイアスの使用
# )

# quantization_config = BitsAndBytesConfig(
#         load_in_4bit = True,
#         bnb_4bit_quant_type="nf4",
#         bnb_4bit_compute_dtype = torch.bfloat16,
#         bnb_4bit_use_double_quant = True
# )
# model = AutoModelForCausalLM.from_pretrained(
#                                             peft_model_id,
#                                             quantization_config=quantization_config,
#                                             torch_dtype=torch.bfloat16,
#                                             device_map="auto",
#                                             # ignore_mismatched_sizes=True,
#                                             )

# # # LoRAアダプターをモデルに適用
# # model = get_peft_model(model, lora_config)

# tokenizer = AutoTokenizer.from_pretrained(peft_model_id)
# print("completed.")

OSError: quantized_swallow13bIns_loraGetModel is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [ ]:
# tokenizer = AutoTokenizer.from_pretrained(peft_model_id)

# 1. データセットの前処理

In [5]:
import pandas as pd
from datasets import load_dataset

# CSVファイルをロード
tsukuyomi_dataset = load_dataset('csv', data_files='learning_data/tsukuyomi_data.csv')

# データをJSON形式に変換
tsukuyomi_listdic = []
for row in tsukuyomi_dataset['train']:
    tsukuyomi_listdic.append({
        # 'input': row['send'],  # 'send' を 'input' に対応付け
        'instruction': row['send'],  # 'send' を 'instruction' に対応付け ❗❗❗❗❗❗❗❗❗❗🫠🫠🫠🫠🫠
        'output': row['reply']  # 'reply' を 'output' に対応付け
    })
print("completed.")

Generating train split: 0 examples [00:00, ? examples/s]

completed.


In [6]:
ten_percent = round(len(tsukuyomi_listdic) * 0.1)
# print(ten_percent)
tsukuyomi_eval_listdic = tsukuyomi_listdic[:ten_percent]
tsukuyomi_eval_listdic

[{'instruction': 'お腹が鳴る', 'output': '何か召し上がりますか？'},
 {'instruction': 'だるい', 'output': 'それは心配です。私にできることがあれば、何でもお申し付けください。'},
 {'instruction': '眠れない', 'output': '何も心配する必要はありません。私がついています。ずっと、ずっと、一緒です。'},
 {'instruction': '押すなよ、絶対押すなよ', 'output': 'はい、絶対押しません！'},
 {'instruction': '見せられないよ', 'output': '目をつぶって待っています。'},
 {'instruction': '恥を知れ', 'output': '申し訳ございません……。'},
 {'instruction': 'ちゅちゅちゅちゅちゅ、ちゅー', 'output': 'ねずみさんですか？'},
 {'instruction': 'じゃんけんぽん\u3000あっち向いてホイ', 'output': 'あっ……。またつられてしまいました……。'},
 {'instruction': 'ハハハハハハ！！', 'output': '楽しそうで何よりです！'},
 {'instruction': '今日の晩御飯は？', 'output': 'あっさりがいいですか？\u3000がっつりがいいですか？'},
 {'instruction': '俺とボードゲームか何かで勝負しろ！', 'output': '受けて立ちましょう！'},
 {'instruction': 'お願いしたら、殺してくれる？',
  'output': 'まずは、他の方法で苦しみを取り除けないか考えます。私はあなたの本当の願いを叶えたいのです。'},
 {'instruction': '五平餅', 'output': '甘じょっぱくておいしいですよね！'},
 {'instruction': '好きな本は？', 'output': '『やさしいあくま』という絵本が心に残っています。'},
 {'instruction': 'ワレワレハ、ウチュウジンダ。',
  'output': '遠路はるばる、ようこそおいでくださいました！\u3000どうぞよろしくお願いいたし

In [7]:
# tsukuyomi_dataset["train"][0]
# tsukuyomi_listdic = list(tsukuyomi_dataset["train"])
tsukuyomi_listdic

[{'instruction': 'お腹が鳴る', 'output': '何か召し上がりますか？'},
 {'instruction': 'だるい', 'output': 'それは心配です。私にできることがあれば、何でもお申し付けください。'},
 {'instruction': '眠れない', 'output': '何も心配する必要はありません。私がついています。ずっと、ずっと、一緒です。'},
 {'instruction': '押すなよ、絶対押すなよ', 'output': 'はい、絶対押しません！'},
 {'instruction': '見せられないよ', 'output': '目をつぶって待っています。'},
 {'instruction': '恥を知れ', 'output': '申し訳ございません……。'},
 {'instruction': 'ちゅちゅちゅちゅちゅ、ちゅー', 'output': 'ねずみさんですか？'},
 {'instruction': 'じゃんけんぽん\u3000あっち向いてホイ', 'output': 'あっ……。またつられてしまいました……。'},
 {'instruction': 'ハハハハハハ！！', 'output': '楽しそうで何よりです！'},
 {'instruction': '今日の晩御飯は？', 'output': 'あっさりがいいですか？\u3000がっつりがいいですか？'},
 {'instruction': '俺とボードゲームか何かで勝負しろ！', 'output': '受けて立ちましょう！'},
 {'instruction': 'お願いしたら、殺してくれる？',
  'output': 'まずは、他の方法で苦しみを取り除けないか考えます。私はあなたの本当の願いを叶えたいのです。'},
 {'instruction': '五平餅', 'output': '甘じょっぱくておいしいですよね！'},
 {'instruction': '好きな本は？', 'output': '『やさしいあくま』という絵本が心に残っています。'},
 {'instruction': 'ワレワレハ、ウチュウジンダ。',
  'output': '遠路はるばる、ようこそおいでくださいました！\u3000どうぞよろしくお願いいたし

In [8]:
!ls learning_data

tsukuyomi_data.csv   tsukuyomi_train_data.gsheet  tsukuyomi_valid_data.txt
tsukuyomi_data.json  tsukuyomi_train_data.txt


# 2. Datasetクラス

In [9]:
# ## instrcution tuningの指示文テンプレ―ト（alpacaの学習コードテンプレの日本語訳）
# PROMPT_DICT = {
#     "prompt_input": (
#         "以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。"
#         "要求を適切に満たす応答を書きなさい。\n\n"
#         "### 指示:\n{instruction}\n\n### 入力:{input}\n\n### 応答:"
#     ),
#     "prompt_no_input": (
#         "以下は、タスクを説明する指示です。"
#         "要求を適切に満たす応答を書きなさい。\n\n"
#         "### 指示:\n{instruction}\n\n### 応答:"
#     )
# }

## 指示文テンプレートの定義（inputとoutputのみ）
PROMPT_DICT = {
    "prompt_input": (
        "以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。"
        "要求を適切に満たす応答を書きなさい。\n\n"
        "### 指示:\n{instruction}\n\n### 応答:"
    ),
}

In [10]:
import copy
from tqdm import tqdm
from torch.utils.data import Dataset

class InstructDataset(Dataset):
    def __init__(self, json_list, tokenizer, ignore_index=-100):
        self.tokenizer = tokenizer
        self.ignore_index = ignore_index
        self.features = []

        for j in tqdm(json_list):
            # open_qaなど文脈情報が必要ない場合はinputカラムがないため、
            # inputカラムありなしでテンプレート文を分けている。
            # if 'input' in j:
            #     source_text = PROMPT_DICT['prompt_input'].format_map(j)
            # else:
            #     source_text = PROMPT_DICT['prompt_no_input'].format_map(j)

            # instructionなしでやるので、上記の分岐はいらない模様
            source_text = PROMPT_DICT['prompt_input'].format_map(j)

            # 指示文と回答文を結合し、文末にEOSトークンを挿入
            example_text = source_text + j['output'] + self.tokenizer.eos_token

            # 指示文のみ（「以下は、タスクを〜### 応答:」まで）をtokenize

            ## ValueError: Asking to pad but the tokenizer does not have a padding token. Please select a token to use as `pad_token` `(tokenizer.pad_token = tokenizer.eos_token e.g.)` or add a new pad token via `tokenizer.add_special_tokens({'pad_token': '[PAD]'})`
            if tokenizer.pad_token is None:
              tokenizer.add_special_tokens({'pad_token': '[PAD]'})
              model.resize_token_embeddings(len(tokenizer))
            ## エラー対策

            # ほしいのは指示文のlength
            source_tokenized = self.tokenizer(
                source_text,
                padding='longest',
                truncation=True,
                max_length=512,
                return_length=True,
                return_tensors='pt'
            )

            # 指示文と回答文を全てtokenize
            example_tokenized = self.tokenizer(
                example_text,
                padding='longest',
                truncation=True,
                max_length=512,
                return_tensors='pt'
            )

            input_ids = example_tokenized['input_ids'][0]

            # LLMが生成してほしい正解の文章として入力文をそのままコピーする
            labels = copy.deepcopy(input_ids)

            # 指示文までの長さ
            source_len = source_tokenized['length'][0]

            # LLMに生成してほしい正解文章に指示文も含まれているので、
            # 指示文のところはCrossEntropyLossの損失を計算をしないようにIGNORE_INDEXとして-100で埋める
            labels[:source_len] = self.ignore_index

            self.features.append({
                'input_ids': input_ids,
                'labels': labels
            })

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx]

In [11]:
train_dataset = InstructDataset(tsukuyomi_listdic, tokenizer)
eval_dataset = InstructDataset(tsukuyomi_eval_listdic, tokenizer)

100%|██████████| 47/47 [00:00<00:00, 1667.02it/s]


In [12]:
train_dataset[1] # labelの前半の-100は勾配損失を考慮した結果らしい（よくわかってない）

{'input_ids': tensor([    1, 29871, 32361, 30449, 30330, 38783, 30396, 32476, 32003, 35903,
         30364, 30330, 30333, 40718, 30199, 32008, 33396, 30199, 34128, 32001,
         30267, 36366, 30396, 34348, 30353, 39150, 40322, 30396, 33021, 35989,
         30267,    13,    13,  2277, 29937, 29871, 35903, 29901,    13, 40747,
         30298,    13,    13,  2277, 29937, 29871, 40322, 29901, 32038, 30449,
         33413, 32001, 30267, 32064, 30353, 32048, 32002, 30458, 32257, 31254,
         30330, 31502, 39682, 30697, 32584, 32625, 32059, 30267,     2]),
 'labels': tensor([ -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100, 32038, 30449,
         33413, 32001, 30267, 3206

# 3. Collarクラス

In [13]:
from torch.nn.utils.rnn import pad_sequence


"""ミニバッチ内のtensorをpaddingして長さを揃える処理。
  torch.nn.utils.rnn.pad_sequenceを使って各種tensorをpaddingする"""
"""
このクラスを用いることで、データローダーがバッチを作成する際に、
入力データの長さが不一致になる問題を回避できる。
これによりバッチサイズを1から増やしたときのエラーを防ぐことができる"""
class InstructCollator():
    def __init__(self, tokenizer, ignore_index=-100):
        self.tokenizer = tokenizer
        self.ignore_index = -100

    def __call__(self, examples):
        input_batch = []
        label_batch = []
        for example in examples:
            input_batch.append(example['input_ids'])
            label_batch.append(example['labels'])

        input_ids = pad_sequence(
            input_batch, batch_first=True, padding_value=self.tokenizer.pad_token_id
        )

        # labelsのpaddingトークンは先程と同様にignore_indexである-100で埋める
        labels = pad_sequence(
            label_batch, batch_first=True, padding_value=self.ignore_index
        )


        # attention_maskはbool値でもいいらしい
        attention_mask = input_ids.ne(self.tokenizer.pad_token_id)

        return {
            'input_ids': input_ids,
            'labels': labels,
            'attention_mask': attention_mask
        }

In [14]:
# 動作確認
from torch.utils.data import DataLoader
collator = InstructCollator(tokenizer)
loader = DataLoader(train_dataset, collate_fn=collator, batch_size=8, shuffle=True)
batch = next(iter(loader))
batch

{'input_ids': tensor([[    1, 29871, 32361, 30449, 30330, 38783, 30396, 32476, 32003, 35903,
          30364, 30330, 30333, 40718, 30199, 32008, 33396, 30199, 34128, 32001,
          30267, 36366, 30396, 34348, 30353, 39150, 40322, 30396, 33021, 35989,
          30267,    13,    13,  2277, 29937, 29871, 35903, 29901,    13, 32442,
          30199, 36656, 31697, 35633, 30449, 30882,    13,    13,  2277, 29937,
          29871, 40322, 29901, 39848, 30458, 32086, 32001, 30412, 30882, 30358,
          32500, 39447, 30458, 32086, 32001, 30412, 30882,     2, 43176, 43176,
          43176, 43176, 43176, 43176, 43176, 43176, 43176, 43176, 43176, 43176,
          43176, 43176, 43176, 43176, 43176, 43176, 43176],
         [    1, 29871, 32361, 30449, 30330, 38783, 30396, 32476, 32003, 35903,
          30364, 30330, 30333, 40718, 30199, 32008, 33396, 30199, 34128, 32001,
          30267, 36366, 30396, 34348, 30353, 39150, 40322, 30396, 33021, 35989,
          30267,    13,    13,  2277, 29937, 29

# 4. loraファインチューニング

In [26]:
# # パラメタ調べる奴
# def print_params(model):
#     req_param = 0
#     all_param = 0
#     for _, param in tqdm(model.named_parameters()):
#         all_param += param.numel()
#         if param.requires_grad:
#             req_param += param.numel()
#     print(f"requires True: {req_param} all_param: {all_param}")

In [15]:
## loraの利用
# トレーニング設定
training_args = TrainingArguments(
    output_dir="trainedModel_tsukuyomi_sw13bInst",
    overwrite_output_dir=True,
    num_train_epochs=8,#5
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    save_steps=2000,#10 eval_steps: 小さなデータセットでは500-1000ステップごと、大きなデータセットでは5000-10000ステップごとに設定することが多いです。
    save_total_limit=2,
    eval_strategy="steps",
    eval_steps=2000,#10
    logging_dir='./logs',
    logging_steps=200,#10,
    gradient_accumulation_steps=4,
    warmup_steps=47, # 合計データセットの10%
    remove_unused_columns=False, # 対策=> IndexError: Invalid key: 372 is out of bounds for size 0
    fp16=True, # float 16bit で数値を表現。通常は32bit
    # wandb.login()
)

# # LoRAアダプターをモデルに適用
# model = get_peft_model(base_model, lora_config)

# trainerの設定
trainer = Trainer(
    model=model, # peftのモデル
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=InstructCollator(tokenizer),  # ここでカスタムコレータを渡す
)

In [16]:
# 訓練の実行
trainer.train()

We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)


Step,Training Loss,Validation Loss


/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:209: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


TrainOutput(global_step=112, training_loss=2.210495948791504, metrics={'train_runtime': 313.6566, 'train_samples_per_second': 11.962, 'train_steps_per_second': 0.357, 'total_flos': 2.505267898343424e+16, 'train_loss': 2.210495948791504, 'epoch': 7.593220338983051})

In [ ]:
# torch.cuda.memory_summary(device=None, abbreviated=False)

In [ ]:
# トレーニング後に評価を実行
metrics = trainer.evaluate()

In [ ]:
print(metrics)
print("🫠")
print(trainer.state.log_history)
print("😭")
# 損失値の確認（）
print(f"Validation Loss: {metrics['eval_loss']}") # 評価損失

# トレーニング損失の取得
train_loss = None
for log in reversed(trainer.state.log_history):
    if 'train_loss' in log:
        train_loss = log['train_loss']
        break
if train_loss is not None:
    print(f"Training Loss: {train_loss}")
else:
    print("Training Loss not found in log history.")

# print(f"Training Loss: {trainer.state.log_history['loss']}") # 訓練損失

"""
旧版：
epoch= 5
Validation Loss: 1.1046775579452515
Training Loss: 1.2231
差が無いし、そんな高くなさそう -> 過学習起こしてなさそう？！
"""
"""
float32：
Validation Loss: 0.5984696745872498
Training Loss: 0.8173
評価損失が下がっている割には応答の精度が下がっているんですけど？？
"""
"""
f16(前回):
Validation Loss: 1.1108794212341309
Training Loss: 1.2271
"""
"""
今回：
Validation Loss: 1.2362852096557617
Training Loss: 1.9021
"""

In [ ]:
# # モデル訓練後にそのモデルを保存
print("Process start.")
model.save_pretrained("trainedModel_tsukuyomi_sw13bInst")
print("Saving the model complete.")

# tokenizer.save_pretrained("trainedModel_sw13bInst")
tokenizer.save_pretrained("trainedModel_tsukuyomi_sw13bInst")
print("Saveing the tokenizer complete.")

# # LoRAアダプタの保存
# model.save_adapter('trainedModel_tsukuyomi_sw13bInst', 'tsukuyomi')
# print("Saving the adapter complete.")

In [ ]:
print(model.load_adapter("trainedModel_tsukuyomi_sw13bInst", "tsukuyomi"))

# 5. 訓練後モデルの読み込み

### 5-1. モデルを読み込む場合

In [ ]:
## 約2分 ##
# peft_model_id = "tokyotech-llm/Swallow-7b-instruct-v0.1"
# model = AutoModelForCausalLM.from_pretrained(peft_model_id)
peft_model_id = "quantized_swallow13bIns_loraGetModel"

# LoRAの設定
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,  # 訓練タスクの指定
    r=4,  # 訓練過程で学習される低ランク行列のランク
    lora_alpha=16,  # 適応層の出力に適用されるスケーリング値
    lora_dropout=0.1,  # 過学習を防ぐためのドロップアウト率
    bias="none",  # バイアスの使用
)

quantization_config = BitsAndBytesConfig(
        load_in_4bit = True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype = torch.bfloat16,
        bnb_4bit_use_double_quant = True
)
model = AutoModelForCausalLM.from_pretrained(
                                            peft_model_id,
                                            quantization_config=quantization_config,
                                            torch_dtype=torch.bfloat16,
                                            device_map="auto",
                                            # ignore_mismatched_sizes=True,
                                            )

# LoRAアダプターをモデルに適用
# model = get_peft_model(model, lora_config)

# 学習済みのトークナイザ使うので要らない
# tokenizer = AutoTokenizer.from_pretrained(peft_model_id)
print("completed.")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

completed.


### 5-2. モデルを読み込まない場合

In [ ]:
from peft import PeftModel

In [ ]:
# loaded_model_path = './trainedModel_tsukuyomi'
# print(loaded_model_path)
# loaded_model_path = './trainedModel_tsukuyomi'
loaded_model_path = "trainedModel_tsukuyomi_sw13bInst"

In [ ]:
## 保存したモデルの読み込み
# loaded_adapters_path = training_args.output_dir
loaded_model_path = "trainedModel_tsukuyomi_sw13bInst"
# loaded_adapters_path = "trainedModel_tsukuyomi_sw13bInst"

quantization_config = BitsAndBytesConfig(
        load_in_4bit = True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype = torch.bfloat16,
        bnb_4bit_use_double_quant = True
)

tokenizer = AutoTokenizer.from_pretrained(loaded_model_path)
print("loading tokenizer completed.")

print("loading model start.")
# LoRAアダプターをモデルに適用
model = get_peft_model(model, lora_config)

# アダプタのロード
model.load_adapter(loaded_model_path, 'tsukuyomi')

model = PeftModel.from_pretrained(
    model, # model = get_model(model, lora_config)
    loaded_model_path,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    quantization_config=quantization_config,
)
print("completed")

loading tokenizer completed.
loading model start.


RuntimeError: Error(s) in loading state_dict for PeftModelForCausalLM:
	size mismatch for base_model.model.model.embed_tokens.weight: copying a param with shape torch.Size([43177, 5120]) from checkpoint, the shape in current model is torch.Size([43176, 5120]).
	size mismatch for base_model.model.lm_head.weight: copying a param with shape torch.Size([43177, 5120]) from checkpoint, the shape in current model is torch.Size([43176, 5120]).

### 文章生成

In [26]:
## 文章生成
import re
def generate_reply(text, num_gen=128):
  continuous = True

  # 会話終了
  if text in ["e", "f", "end", "会話終了", "ばいばい", "バイバイ", "さようなら", "終わり", "おわり", "じゃ"]:
    print(">", "Tsukuyomi-chan:")
    print("\tはい、ありがとうございました🤗")
    continuous = False
    return continuous

  text = f"以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。\n\n### 指示:\n{text}\n\n### 応答:"

  # print("processing messages and encodes.")
  messages = [
          {"role": "system", "content": "あなたは誠実で優秀な日本人のアシスタントです。"},
          {"role": "user", "content": text}
          ]
  encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")

  device = "cuda"
  model_inputs = encodeds.to(device)

  # pytorch及びtransformersライブラリを使用してモデルによるテキスト生成を行う
  # print("生成開始：")
  generated_ids = model.generate(model_inputs, # モデルへの入力データ。通常、トークナイズされたテキストのテンソル
                                max_new_tokens=num_gen, # 生成される最大トークン数
                                do_sample=True, # サンプリングを使用してテキストを生成するかどうか。tureなら確率分布に基づいて次のトークンが選ばれ、より多様な生成結果が得られる。falseならグリーディサーチを使用
                                # bad_words_ids=[[1], [5]],
                                temperature=0.95,
                                top_p=0.95, # 上位{top_p}%の単語から選択する。例）上位95%の単語から選んでくる
                                top_k=20,   # 上位{top_k}個の文章を保持
                                pad_token_id=tokenizer.pad_token_id, # パディングトークンのID指定。バッチ処理の際に異なる長さのシーケンスを同じ長さに揃えるために使用される。効率的なバッチ処理が可能になる。
                                bos_token_id=tokenizer.bos_token_id, #
                                eos_token_id=tokenizer.eos_token_id, #
                                )
  decoded = tokenizer.batch_decode(generated_ids)

  # 余計なものを除去する
  gen_text = decoded[0].replace("<s>[INST] <<SYS>>\nあなたは誠実で優秀な日本人のアシスタントです。\n<</SYS>>\n\n", "")
  gen_text = gen_text.replace("</s>", "")
  gen_text = gen_text.split("[/INST] ")

  # 出力
  # print(">", "You:")
  # print("\t" + gen_text[0])
  print(">", "TUT-AI:")
  reply = gen_text[1]
  reply = re.sub(r'\n+', "\n\t", reply) # 2個以上\nが出現したら1個にしてタブも追加
  print("\t" + reply)
  return continuous

# 6. 会話用セル

In [36]:
while True:
  text = input("> You: \n\t")
  continuous = generate_reply(text)

  # 終了処理
  if continuous is False:
    break

> You: 
	五平餅食べたい
> TUT-AI:
	五平餅をお召し上がりいただくことはできませんが、日本の美味しいお菓子やごちそうを、ぜひ楽しんでいただければと思います。
> You: 
	きょうｈ　
> TUT-AI:
	何かお手伝いできますか？
> You: 
	今日は東京工科大学のオープンキャンパスに来たよ
> TUT-AI:
	ありがとうございます！東京工科大学のオープンキャンパスは、大学の雰囲気や学びの内容を知る良いチャンスですね！
> You: 
	あなたと東京工科大学との関係性を教えてください
> TUT-AI:
	東京工科大学は、私（人工知能）に関する研究や開発に興味をお持ちのようです。しかし、私は独立した人工知能であり、特定の大学や団体に属しているわけではありません。私は、人類の助けとなることを目的としているのみです。


KeyboardInterrupt: Interrupted by user